In [1]:
import numpy as np
from scipy.stats import entropy
from sklearn.metrics import mutual_info_score
# Dependecias del Notebook
import pandas as pd
import numpy as np
import pandas_datareader.data as data_web
import datetime as dt
import matplotlib.pyplot as plt
import plotly.express as px
import scipy
import scipy.spatial
from scipy.spatial.distance import directed_hausdorff
import yfinance as yf
from numpy.fft import fft, ifft
from statsmodels.tsa.stattools import coint
import yfinance as yf
import datetime
import seaborn as sns
from scipy.stats import entropy
from sklearn.metrics import mutual_info_score
from Dicts import Dicts

In [2]:
dicts = Dicts()

c:\Users\lgarcia\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
stocks_dict = dicts.stocks_dict()
etf_list = dicts.etf_dict()
forex_list = dicts.forex_dict()
crypto_list = dicts.crypto_dict()

In [4]:
def correlation_distance(X, Y):
    cor = np.corrcoef(X,Y)[0,1]
    metrica = np.sqrt(0.5*(1-cor))
    return metrica

def varInfo(X,Y,bins,norm=False):
    import numpy as np
    # variation of information
    cXY=np.histogram2d(X,Y,bins)[0]
    iXY=mutual_info_score(None,None,contingency=cXY)
    hX=entropy(np.histogram(X,bins)[0]) # marginal
    hY=entropy(np.histogram(Y,bins)[0]) # marginal
    vXY=hX+hY-2*iXY # variation of information
    if norm:
        hXY=hX+hY-iXY # joint
        vXY/=hXY # normalized variation of information
    return vXY

def Descargar(start,end,stock_type,sector,number,show):
    diccionarioStocks = {
'Stock': stocks_dict }

    if stock_type != 'Stock':
        if number < len(diccionarioStocks[stock_type]):
            tickers = np.random.permutation(diccionarioStocks[stock_type])[:number]
        else:
            print('El numero de tickers que escogiste es mayor al que tenemos, así que te muestro todos')
            tickers = np.random.permutation(diccionarioStocks[stock_type])

    else: 
        if  number < len(diccionarioStocks[stock_type]):
            tickers = (np.random.permutation(diccionarioStocks[stock_type][sector])[:number]).tolist()
        else:
            print('El numero de tickers que escogiste es mayor al que tenemos, así que te muestro todos')
            tickers = np.random.permutation(diccionarioStocks[stock_type][sector])
            
    #tickers=diccionarioStocks[stock_type][sector]
    df = yf.download(tickers, start, end)['Close'].dropna()
    pairs = []
    for i in range(len(tickers)):
        for j in range(i+1,len(tickers)):
            pairs.append((tickers[i],tickers[j]))
    if show=="Data Frame":
        return df
    elif show=="Pairs":
        return pairs

In [9]:
class paresTrading:
    def __init__(self,start,end,stock_type,sector,number):
        diccionarioStocks = { 'Crypto':crypto_list, 'Forex': forex_list, 'Stock': stocks_dict, "Etf": etf_list}
        if stock_type != 'Stock':
            if number < len(diccionarioStocks[stock_type]):
                self.tickers = np.random.permutation(diccionarioStocks[stock_type])[:number].tolist()
            else:
                print('El numero de tickers que escogiste es mayor al que tenemos, así que te muestro todos')
                self.tickers = np.random.permutation(diccionarioStocks[stock_type])

        else: 
            if  number < len(diccionarioStocks[stock_type]):
                self.tickers = (np.random.permutation(diccionarioStocks[stock_type][sector])[:number]).tolist()
            else:
                print('El numero de tickers que escogiste es mayor al que tenemos, así que te muestro todos')
                self.tickers = np.random.permutation(diccionarioStocks[stock_type][sector]).tolist()

        self.start = start
        self.end = end
        #tickers=diccionarioStocks[stock_type][sector]
        self.df = yf.download(self.tickers, start, end)['Close'].dropna()
        self.data_filtration = self.df.copy()
        self.pairs = []
        for i in range(len(self.tickers)):
            for j in range(i+1,len(self.tickers)):
                self.pairs.append((self.tickers[i],self.tickers[j]))
                
    def correlationTest(self, tol = 0.9):
        for pair in self.pairs:
                resultado = np.corrcoef(self.data_filtration[pair[0]],self.data_filtration[pair[1]])[0,1]
                if resultado < tol:
                    self.pairs.remove(pair) 
        self.__filter_data()
        
    def cointegration(self, tol=0.1):
        for pair in self.pairs:
                resultado = coint(self.data_filtration[pair[0]], self.data_filtration[pair[1]])[1]
                if resultado > tol:
                    self.pairs.remove(pair)  
        self.__filter_data()
        
    def distance_correlationTest(self, tol=0.3):
        for pair in self.pairs:
            resultado = correlation_distance(self.data_filtration[pair[0]], self.data_filtration[pair[1]])
            if resultado > tol:
                self.pairs.remove(pair)    
        self.__filter_data()
   
                
    def variationalInformationTest(self, bins = 50, tol = 3.0):
        for pair in self.pairs:
            resultado = varInfo(self.data_filtration[pair[0]],self.data_filtration[pair[1]], bins = bins)
            if resultado > tol:
                self.pairs.remove(pair) 
        self.__filter_data()

    def clear_filter(self):
        self.pairs = []
        self.data_filtration = self.df.copy()
        for i in range(len(self.tickers)):
            for j in range(i+1,len(self.tickers)):
                self.pairs.append((self.tickers[i],self.tickers[j]))
            
  
    def __filter_data(self):
        unique_tickers = []
        for pair in self.pairs:
            if pair[0] not in unique_tickers:
                unique_tickers.append(pair[0])
            if pair[1] not in unique_tickers:
                unique_tickers.append(pair[1])       
        self.data_filtration = self.data_filtration[unique_tickers]
        
    
    def plot_correlation(self, method = 'pearson', *args):
        plt.figure(figsize=(25,25))
        sns.heatmap(self.data_filtration.corr(method = method), cmap = sns.cm.rocket, *args)
        
    def plot_variational_information(self, bins= 50, *args):
        varinfo_matrix = np.zeros([self.data_filtration.shape[1], self.data_filtration.shape[1]])
        for i in range(self.data_filtration.shape[1]):
            for j in range(self.data_filtration.shape[1]):
                varinfo_matrix[i,j] = varInfo(self.data_filtration.iloc[:,i], self.data_filtration.iloc[:,j], bins = 50)
        varinfo_matrix = pd.DataFrame(varinfo_matrix, columns=self.data_filtration.columns)
        
        plt.figure(figsize=(25,25))
        sns.heatmap(varinfo_matrix, cmap = sns.cm.rocket, xticklabels=self.tickers, yticklabels=self.tickers, *args)
        
    def plot_correlationDistance(self, method = 'pearson', *args):
        
        cor = self.data_filtration.corr(method='pearson')
        metrica = np.sqrt(0.5*(1-cor))
        plt.figure(figsize=(25,25))
        sns.heatmap(metrica, cmap = sns.cm.rocket_r)


In [12]:
start="2022-01-01"
end="2022-06-01"
modelo=paresTrading(start=start,end=end,stock_type="Forex",sector='Consumer Defensive',number=6)

[*********************100%***********************]  6 of 6 completed

6 Failed downloads:
- TOPUSD: No data found, symbol may be delisted
- HTGUSD: No data found, symbol may be delisted
- YERUSD: No data found, symbol may be delisted
- CHFUSD: No data found, symbol may be delisted
- SCRUSD: No data found, symbol may be delisted
- BSDUSD: No data found, symbol may be delisted


In [17]:
diccionarioStocks = {"Stock": stocks_dict}

In [18]:
diccionarioStocks["Stock"]["Consumer Defensive"]

['PG',
 'WMT',
 'KO',
 'PEP',
 'COST',
 'PM',
 'UL',
 'DEO',
 'BTI',
 'BUD',
 'EL',
 'MDLZ',
 'MO',
 'FMX',
 'CL',
 'TGT',
 'DG',
 'KDP',
 'MNST',
 'KHC',
 'KMB',
 'GIS',
 'HSY',
 'STZ-B',
 'STZ',
 'SYY',
 'ADM',
 'DLTR',
 'KR',
 'BF-B',
 'BF-A',
 'TSN',
 'HRL',
 'CCEP',
 'K',
 'CHD',
 'MKC',
 'MKC-V',
 'CLX',
 'CAG',
 'ACI',
 'CPB',
 'SJM',
 'BG',
 'TAP-A',
 'TAP',
 'KOF',
 'LW',
 'DAR',
 'BJ',
 'NWL',
 'PPC',
 'PFGC',
 'USFD',
 'COTY',
 'INGR',
 'FLO',
 'COKE',
 'POST',
 'SAFM',
 'FIZZ',
 'GO',
 'HELE',
 'SAM',
 'ASAI',
 'LANC',
 'NOMD',
 'NTCO',
 'EDU',
 'BRBR',
 'SPB',
 'LOPE',
 'GHC',
 'SFM',
 'JJSF',
 'CALM',
 'CHGG',
 'TR',
 'THS',
 'IPAR',
 'HLF',
 'CCU',
 'UNFI',
 'NAPA',
 'DNUT',
 'CENTA',
 'CENT',
 'PSMT',
 'NUS',
 'MGPI',
 'PRMW',
 'IBA',
 'HAIN',
 'COUR',
 'LAUR',
 'WMK',
 'EPC',
 'LRN',
 'STRA',
 'IMKTA',
 'BGS',
 'AKO-B',
 'ATGE',
 'VGR',
 'BYND',
 'AKO-A',
 'SOVO',
 'ELF',
 'UVV',
 'CHEF',
 'FDP',
 'USNA',
 'UDMY',
 'DDL',
 'UTZ',
 'EWCZ',
 'ANDE',
 'SPTN',
 'AVO',
 'TW

In [16]:
modelo.clear_filter()
modelo.distance_correlationTest(tol=0.5)
modelo.variationalInformationTest(tol = 1.0)
modelo.cointegration(tol=)
#modelo.topo(tol=10)

In [14]:
modelo.pairs

[('TGT', 'PG'), ('PEP', 'PG'), ('MO', 'PG')]

In [12]:
modelo.pairs

[('TGT', 'PG'), ('KO', 'MO')]